In [3]:
import gcsfs
import dask_gateway
import dask.dataframe as dd
import pandas as pd
import json

In [77]:
token = json.load(open("credentials.json"))
fs = gcsfs.GCSFileSystem(token=token)
storage_options={"token": token}

In [116]:
files = [f"gcs://{f}" for f in fs.glob("quansight-datasets/airline-ontime-performance/csv/*.csv")]

In [84]:
gateway = dask_gateway.Gateway()

In [11]:
options = gateway.cluster_options(use_local_defaults=False)
options

In [12]:
cluster = gateway.new_cluster(options)
cluster

In [13]:
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://nebari.quansight.dev/gateway/clusters/dev.8146904d4ea54dc897d1f7e1a0fec7bf/status,


In [4]:
with open('dtypes.json', 'r') as f:
    dtypes = json.load(f)

In [85]:
df = pd.read_csv(files[0], dtype=dtypes, storage_options=storage_options)

In [61]:
files = [f"gcs://{f}" for f in fs.glob("quansight-datasets/airline-ontime-performance/csv/*2022*.csv")]

In [92]:
ddf = dd.read_csv("gcs://quansight-datasets/airline-ontime-performance/csv/*", 
                  dtype=dtypes, 
                  parse_dates=['FL_DATE'], 
                  storage_options=storage_options,
                 )

In [93]:
ddf.describe().compute()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,...,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME
count,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,1.256803e+08,...,18.000000,18.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2.012097e+03,2.508042e+00,6.516893e+00,1.563974e+01,3.942376e+00,1.998057e+04,2.277859e+03,1.266671e+04,1.266674e+06,3.174295e+04,...,10.166667,9.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5.816493e+00,1.113230e+00,3.421065e+00,8.761722e+00,1.992835e+00,3.895443e+02,1.889318e+03,1.525418e+03,1.525416e+05,1.308327e+03,...,14.496450,10.702147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.003000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.938600e+04,0.000000e+00,1.013500e+04,1.013501e+06,3.007000e+04,...,3.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.007000e+03,2.000000e+00,4.000000e+00,9.000000e+00,3.000000e+00,1.979000e+04,1.265000e+03,1.129800e+04,1.129802e+06,3.072100e+04,...,4.250000,4.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.012000e+03,3.000000e+00,7.000000e+00,1.600000e+01,5.000000e+00,2.030400e+04,2.518000e+03,1.289200e+04,1.289208e+06,3.145400e+04,...,6.000000,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.017000e+03,3.000000e+00,9.000000e+00,2.400000e+01,6.000000e+00,2.037800e+04,4.629000e+03,1.410000e+04,1.410005e+06,3.257500e+04,...,10.000000,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2.022000e+03,4.000000e+00,1.200000e+01,3.100000e+01,7.000000e+00,2.117100e+04,9.912000e+03,1.686900e+04,1.686902e+06,3.613300e+04,...,66.000000,49.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
ddf.groupby(['MONTH']).count().compute()

,YEAR,QUARTER,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
MONTH,,,,,,,,,,,,,,,,,,,,,
4,556502,556502,556502,556502,556502,556502,556502,556502,554935,556502,...,0,0,0,0,0,0,0,0,0,0
8,589810,589810,589810,589810,589810,589810,589810,589810,588352,589810,...,0,0,0,0,0,0,0,0,0,0
2,495713,495713,495713,495713,495713,495713,495713,495713,491714,495713,...,0,0,0,0,0,0,0,0,0,0
1,537902,537902,537902,537902,537902,537902,537902,537902,529429,537902,...,0,0,0,0,0,0,0,0,0,0
7,594957,594957,594957,594957,594957,594957,594957,594957,593851,594957,...,0,0,0,0,0,0,0,0,0,0
6,577283,577283,577283,577283,577283,577283,577283,577283,574457,577283,...,0,0,0,0,0,0,0,0,0,0
3,564853,564853,564853,564853,564853,564853,564853,564853,563090,564853,...,0,0,0,0,0,0,0,0,0,0
5,578819,578819,578819,578819,578819,578819,578819,578819,577483,578819,...,0,0,0,0,0,0,0,0,0,0
11,546410,546410,546410,546410,546410,546410,546410,546410,545499,546410,...,0,0,0,0,0,0,0,0,0,0


In [96]:
ddf.to_parquet("gcs://quansight-datasets/airline-ontime-performance/parquet_by_year", 
            partition_on=['YEAR'], 
            storage_options=storage_options,
            write_index=False,
            )

In [101]:
fs.ls("gcs://quansight-datasets/airline-ontime-performance/full_dataset.parquet")

['quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.0.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.10.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.100.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1000.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1001.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1002.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1003.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1004.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1005.parquet',
 'quansight-datasets/airline-ontime-performance/full_dataset.parquet/part.1006.parquet',
 'quansight-datasets/airline-o

In [98]:
ddf.repartition(partition_size="100MB").to_parquet("gcs://quansight-datasets/airline-ontime-performance/full_dataset.parquet", 
            storage_options=storage_options,
            write_index=False,
            )

In [102]:
ddf2 = dd.read_parquet("gcs://quansight-datasets/airline-ontime-performance/full_dataset.parquet", 
            storage_options=storage_options)

In [115]:
ddf2['FL_DATE'].describe().compute()

/home/conda/nebari-git/42d83ad8675b5508107c3f02d91e89f5d004f397e7535cbe02760cdf35581e77-20230104-200630-498497-33-dask/lib/python3.10/site-packages/dask/dataframe/core.py:2842: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  meta = self._meta_nonempty.describe(
/home/conda/nebari-git/42d83ad8675b5508107c3f02d91e89f5d004f397e7535cbe02760cdf35581e77-20230104-200630-498497-33-dask/lib/python3.10/site-packages/dask/dataframe/core.py:3041: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  meta = data._meta_nonempty.describe(**datetime_is_numeric_kwarg)


unique                   7178
count               125680321
top       2019-12-01 00:00:00
freq                    22784
first     2003-01-01 00:00:00
last      2022-11-30 00:00:00
Name: FL_DATE, dtype: object